In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [3]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_gu.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,291784,2339747209,4/16/20,951,PPP,GLOBAL FOOD SERVICES LLC,415 Chalan San Antonio Ste 306,TAMUNING,GU,96913.0,...,999 Other,5614 Business Support Services,5614.0,HAWAII DISTRICT OFFICE,d $2-5 million,500 or more,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,NaN
1,291785,3868457205,4/27/20,951,PPP,"TANOTA DEVELOPMENT, LLC",1227 PALE SAN VITORES RD,TAMUNING,GU,96913.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN
2,291786,6199037203,4/27/20,951,PPP,CITY HILL COMPANY (GUAM) LTD.,1328 PALE SAN VITORES RD,TAMUNING,GU,96913.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,NaN
3,291787,4008007204,4/27/20,951,PPP,"BAYVIEW II, LLC",1227 PALE SAN VITORES RD,TAMUNING,GU,96913.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN
4,291788,1723687901,6/10/20,951,PPP,LEOPALACE GUAM CORPORATION,245 La Cuesta CR 245 La Cuesta CR,YONA,GU,96915.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,292222,4259917200,4/27/20,951,PPP,MERCY HEIGHTS NURSERY AND KINDERGARTEN,211 Fr. San Vitores Street,Perezville-Tamuning,GU,96913.0,...,999 Other,6244 Child Day Care Services,6244.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"211 Fr. San Vitores Street, Perezville-Tamunin...",13.515673,144.806226,NaN
439,292223,9938928204,1/14/21,951,PPS,"INFRATECH INTERNATIONAL, LLC",122 N. Chalan Okso Road,Harmon,GU,96913.0,...,999 Other,2371 Utility System Construction,2371.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"122 N. Chalan Okso Road, Harmon, GU",13.539564,144.873940,NaN
440,292224,1002328807,4/9/21,951,PPS,PACIFIC METAL WORKS CORPORATION,165F Guerrero St Unit 5,Tamuning,GU,96913.0,...,332 Fabricated Metal Products,3323 Architectural and Structural Metals Manuf...,3323.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"165F Guerrero St Unit 5, Tamuning, GU",13.491804,144.785152,NaN
441,292225,2478887205,4/16/20,951,PPP,"REALTY MANAGEMENT COMPANY, INC.",701 S MARINE CORPS DR,TAMUNING,GU,96913.0,...,999 Other,5313 Activities Related to Real Estate,5313.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"701 S MARINE CORPS DR, TAMUNING, GU",13.486372,144.780195,NaN


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [5]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [6]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,2339747209,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474
1,3868457205,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999
2,6199037203,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437
3,4008007204,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999
4,1723687901,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../data/state_data/geo/geo_fips/GU_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [27]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [7]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [12]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
#executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('66')] #GU
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

The version of chrome cannot be detected. Trying with latest driver version


  0%|          | 0/443 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [13]:
geo_id_list =geo_id 

Length Check

In [14]:
print(len(geo_id_list))

443


Matching FIPS to DataFrame

In [15]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [16]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,2339747209,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,[660109522001001]
1,3868457205,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,[660109519024005]
2,6199037203,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,[660109519024005]
3,4008007204,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,[660109519024005]
4,1723687901,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,[660109539004001]
...,...,...,...,...,...
438,4259917200,"211 Fr. San Vitores Street, Perezville-Tamunin...",13.515673,144.806226,[660109519024004]
439,9938928204,"122 N. Chalan Okso Road, Harmon, GU",13.539564,144.873940,[660109507023008]
440,1002328807,"165F Guerrero St Unit 5, Tamuning, GU",13.491804,144.785152,[660109523003004]
441,2478887205,"701 S MARINE CORPS DR, TAMUNING, GU",13.486372,144.780195,[660109522004004]


Remove brackets

In [17]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,2339747209,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,660109522001001
1,3868457205,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,660109519024005
2,6199037203,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,660109519024005
3,4008007204,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,660109519024005
4,1723687901,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,660109539004001
...,...,...,...,...,...
438,4259917200,"211 Fr. San Vitores Street, Perezville-Tamunin...",13.515673,144.806226,660109519024004
439,9938928204,"122 N. Chalan Okso Road, Harmon, GU",13.539564,144.873940,660109507023008
440,1002328807,"165F Guerrero St Unit 5, Tamuning, GU",13.491804,144.785152,660109523003004
441,2478887205,"701 S MARINE CORPS DR, TAMUNING, GU",13.486372,144.780195,660109522004004


In [18]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
71,5837648304,"LOT 439-1-2 ROUTE 2A, AGAT, GU",42.447285,-71.265049,NaN


Parse for Merge

In [19]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,2339747209,660109522001001
1,3868457205,660109519024005
2,6199037203,660109519024005
3,4008007204,660109519024005
4,1723687901,660109539004001
...,...,...
438,4259917200,660109519024004
439,9938928204,660109507023008
440,1002328807,660109523003004
441,2478887205,660109522004004


Read in base data

In [21]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_gu.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/GU_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,291784,291784,291784,2339747209,4/16/20,951,PPP,GLOBAL FOOD SERVICES LLC,415 Chalan San Antonio Ste 306,TAMUNING,...,999 Other,5614 Business Support Services,5614.0,HAWAII DISTRICT OFFICE,d $2-5 million,500 or more,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,NaN
1,291785,291785,291785,3868457205,4/27/20,951,PPP,"TANOTA DEVELOPMENT, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN
2,291786,291786,291786,6199037203,4/27/20,951,PPP,CITY HILL COMPANY (GUAM) LTD.,1328 PALE SAN VITORES RD,TAMUNING,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,NaN
3,291787,291787,291787,4008007204,4/27/20,951,PPP,"BAYVIEW II, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN
4,291788,291788,291788,1723687901,6/10/20,951,PPP,LEOPALACE GUAM CORPORATION,245 La Cuesta CR 245 La Cuesta CR,YONA,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,NaN


Merge with MainFrame

In [22]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,291784,291784,291784,2339747209,4/16/20,951,PPP,GLOBAL FOOD SERVICES LLC,415 Chalan San Antonio Ste 306,TAMUNING,...,5614 Business Support Services,5614.0,HAWAII DISTRICT OFFICE,d $2-5 million,500 or more,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,NaN,660109522001001
1,291785,291785,291785,3868457205,4/27/20,951,PPP,"TANOTA DEVELOPMENT, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN,660109519024005
2,291786,291786,291786,6199037203,4/27/20,951,PPP,CITY HILL COMPANY (GUAM) LTD.,1328 PALE SAN VITORES RD,TAMUNING,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,NaN,660109519024005
3,291787,291787,291787,4008007204,4/27/20,951,PPP,"BAYVIEW II, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN,660109519024005
4,291788,291788,291788,1723687901,6/10/20,951,PPP,LEOPALACE GUAM CORPORATION,245 La Cuesta CR 245 La Cuesta CR,YONA,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,NaN,660109539004001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,292222,292222,292222,4259917200,4/27/20,951,PPP,MERCY HEIGHTS NURSERY AND KINDERGARTEN,211 Fr. San Vitores Street,Perezville-Tamuning,...,6244 Child Day Care Services,6244.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"211 Fr. San Vitores Street, Perezville-Tamunin...",13.515673,144.806226,NaN,660109519024004
439,292223,292223,292223,9938928204,1/14/21,951,PPS,"INFRATECH INTERNATIONAL, LLC",122 N. Chalan Okso Road,Harmon,...,2371 Utility System Construction,2371.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"122 N. Chalan Okso Road, Harmon, GU",13.539564,144.873940,NaN,660109507023008
440,292224,292224,292224,1002328807,4/9/21,951,PPS,PACIFIC METAL WORKS CORPORATION,165F Guerrero St Unit 5,Tamuning,...,3323 Architectural and Structural Metals Manuf...,3323.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"165F Guerrero St Unit 5, Tamuning, GU",13.491804,144.785152,NaN,660109523003004
441,292225,292225,292225,2478887205,4/16/20,951,PPP,"REALTY MANAGEMENT COMPANY, INC.",701 S MARINE CORPS DR,TAMUNING,...,5313 Activities Related to Real Estate,5313.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"701 S MARINE CORPS DR, TAMUNING, GU",13.486372,144.780195,NaN,660109522004004


<b>Fill FIPS Columns - if FIPS file

In [23]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,291784,291784,291784,2339747209,4/16/20,951,PPP,GLOBAL FOOD SERVICES LLC,415 Chalan San Antonio Ste 306,TAMUNING,...,5614.0,HAWAII DISTRICT OFFICE,d $2-5 million,500 or more,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,NaN,660109522001001,660109522001001
1,291785,291785,291785,3868457205,4/27/20,951,PPP,"TANOTA DEVELOPMENT, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN,660109519024005,660109519024005
2,291786,291786,291786,6199037203,4/27/20,951,PPP,CITY HILL COMPANY (GUAM) LTD.,1328 PALE SAN VITORES RD,TAMUNING,...,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,NaN,660109519024005,660109519024005
3,291787,291787,291787,4008007204,4/27/20,951,PPP,"BAYVIEW II, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,NaN,660109519024005,660109519024005
4,291788,291788,291788,1723687901,6/10/20,951,PPP,LEOPALACE GUAM CORPORATION,245 La Cuesta CR 245 La Cuesta CR,YONA,...,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,NaN,660109539004001,660109539004001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,292222,292222,292222,4259917200,4/27/20,951,PPP,MERCY HEIGHTS NURSERY AND KINDERGARTEN,211 Fr. San Vitores Street,Perezville-Tamuning,...,6244.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"211 Fr. San Vitores Street, Perezville-Tamunin...",13.515673,144.806226,NaN,660109519024004,660109519024004
439,292223,292223,292223,9938928204,1/14/21,951,PPS,"INFRATECH INTERNATIONAL, LLC",122 N. Chalan Okso Road,Harmon,...,2371.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"122 N. Chalan Okso Road, Harmon, GU",13.539564,144.873940,NaN,660109507023008,660109507023008
440,292224,292224,292224,1002328807,4/9/21,951,PPS,PACIFIC METAL WORKS CORPORATION,165F Guerrero St Unit 5,Tamuning,...,3323.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"165F Guerrero St Unit 5, Tamuning, GU",13.491804,144.785152,NaN,660109523003004,660109523003004
441,292225,292225,292225,2478887205,4/16/20,951,PPP,"REALTY MANAGEMENT COMPANY, INC.",701 S MARINE CORPS DR,TAMUNING,...,5313.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"701 S MARINE CORPS DR, TAMUNING, GU",13.486372,144.780195,NaN,660109522004004,660109522004004


Drop Extra FIPS Columns

In [24]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,291784,291784,291784,2339747209,4/16/20,951,PPP,GLOBAL FOOD SERVICES LLC,415 Chalan San Antonio Ste 306,TAMUNING,...,999 Other,5614 Business Support Services,5614.0,HAWAII DISTRICT OFFICE,d $2-5 million,500 or more,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,660109522001001
1,291785,291785,291785,3868457205,4/27/20,951,PPP,"TANOTA DEVELOPMENT, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,660109519024005
2,291786,291786,291786,6199037203,4/27/20,951,PPP,CITY HILL COMPANY (GUAM) LTD.,1328 PALE SAN VITORES RD,TAMUNING,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,660109519024005
3,291787,291787,291787,4008007204,4/27/20,951,PPP,"BAYVIEW II, LLC",1227 PALE SAN VITORES RD,TAMUNING,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,660109519024005
4,291788,291788,291788,1723687901,6/10/20,951,PPP,LEOPALACE GUAM CORPORATION,245 La Cuesta CR 245 La Cuesta CR,YONA,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,660109539004001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,292222,292222,292222,4259917200,4/27/20,951,PPP,MERCY HEIGHTS NURSERY AND KINDERGARTEN,211 Fr. San Vitores Street,Perezville-Tamuning,...,999 Other,6244 Child Day Care Services,6244.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"211 Fr. San Vitores Street, Perezville-Tamunin...",13.515673,144.806226,660109519024004
439,292223,292223,292223,9938928204,1/14/21,951,PPS,"INFRATECH INTERNATIONAL, LLC",122 N. Chalan Okso Road,Harmon,...,999 Other,2371 Utility System Construction,2371.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"122 N. Chalan Okso Road, Harmon, GU",13.539564,144.873940,660109507023008
440,292224,292224,292224,1002328807,4/9/21,951,PPS,PACIFIC METAL WORKS CORPORATION,165F Guerrero St Unit 5,Tamuning,...,332 Fabricated Metal Products,3323 Architectural and Structural Metals Manuf...,3323.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"165F Guerrero St Unit 5, Tamuning, GU",13.491804,144.785152,660109523003004
441,292225,292225,292225,2478887205,4/16/20,951,PPP,"REALTY MANAGEMENT COMPANY, INC.",701 S MARINE CORPS DR,TAMUNING,...,999 Other,5313 Activities Related to Real Estate,5313.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"701 S MARINE CORPS DR, TAMUNING, GU",13.486372,144.780195,660109522004004


In [25]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
71,291855,291855,291855,5837648304,1/25/21,951,PPS,AIC INTERNATIONAL INC.,LOT 439-1-2 ROUTE 2A,AGAT,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"LOT 439-1-2 ROUTE 2A, AGAT, GU",42.447285,-71.265049,NaN


Convert to File

In [26]:
fips_GU = fips_merge_drop

In [28]:
fips_GU.to_csv("../../data/state_data/geo/geo_fips/GU_fips_scraped.csv") 

Review & Compare

In [29]:
geo_GU_fips = pd.read_csv("../../data/state_data/geo/geo_fips/GU_fips_scraped.csv") 
geo_GU_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,291784,291784,291784,2339747209,4/16/20,951,PPP,GLOBAL FOOD SERVICES LLC,415 Chalan San Antonio Ste 306,...,999 Other,5614 Business Support Services,5614.0,HAWAII DISTRICT OFFICE,d $2-5 million,500 or more,"415 Chalan San Antonio Ste 306, TAMUNING, GU",13.496740,144.780474,6.601095e+14
1,1,291785,291785,291785,3868457205,4/27/20,951,PPP,"TANOTA DEVELOPMENT, LLC",1227 PALE SAN VITORES RD,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,6.601095e+14
2,2,291786,291786,291786,6199037203,4/27/20,951,PPP,CITY HILL COMPANY (GUAM) LTD.,1328 PALE SAN VITORES RD,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1328 PALE SAN VITORES RD, TAMUNING, GU",13.515498,144.807437,6.601095e+14
3,3,291787,291787,291787,4008007204,4/27/20,951,PPP,"BAYVIEW II, LLC",1227 PALE SAN VITORES RD,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"1227 PALE SAN VITORES RD, TAMUNING, GU",13.513656,144.805999,6.601095e+14
4,4,291788,291788,291788,1723687901,6/10/20,951,PPP,LEOPALACE GUAM CORPORATION,245 La Cuesta CR 245 La Cuesta CR,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,250 to 499,"245 La Cuesta CR 245 La Cuesta CR, YONA, GU",13.400996,144.752189,6.601095e+14


In [30]:
##### fill in missing Lat, Long, FIPS
geo_GU_fips.loc[geo_GU_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
71,71,291855,291855,291855,5837648304,1/25/21,951,PPS,AIC INTERNATIONAL INC.,LOT 439-1-2 ROUTE 2A,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"LOT 439-1-2 ROUTE 2A, AGAT, GU",42.447285,-71.265049,NaN


In [32]:
geo_GU_fips["BusinessType"] = geo_GU_fips["BusinessType"].fillna('Not Available')
geo_GU_fips["BusinessType"].unique()
geo_GU_fips.to_csv("../../data/state_data/geo/geo_fips/GU_fips_scraped.csv") 
geo_GU_fips =pd.read_csv("../../data/state_data/geo/geo_fips/GU_fips_scraped.csv") 
geo_GU_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Corporation',
       'Subchapter S Corporation', 'Non-Profit Organization',
       'Limited Liability Partnership', 'Sole Proprietorship',
       'Partnership', 'Professional Association',
       'Non-Profit Childcare Center'], dtype=object)